In [12]:
from hyperopt.pyll.base import scope 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from tensorflow.keras import regularizers


In [62]:
# importing all the necessary libraries!

import os
import warnings
import json
from datetime import datetime 
import pandas as pd
import numpy as np
from math import sqrt
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.regularizers import L1L2
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from keras.constraints import maxnorm
from hyperopt.pyll.base import scope 

In [71]:
train_X = np.array([[[1,2],[3,4],[5,6]],[[1,2],[3,4],[5,6]],[[1,2],[3,4],[5,6]],[[1,2],[3,4],[5,6]]])
train_X.shape

train_y = np.array([[1,3],[1,3],[1,3],[1,3]])
#train_y.shape[1]

In [72]:
space = {'rate'       : hp.uniform('rate',0.01,0.5),
         'units'      : scope.int(hp.quniform('units',10,100,5)),
         'batch_size' : scope.int(hp.quniform('batch_size',100,250,25)),
         'layers'     : scope.int(hp.quniform('layers',1,6,1)),
         'dropout'     : scope.int(hp.quniform('dropout',0,0.75,0.25))
        }

In [73]:
def build_model(space): #train_data_X.shape[1], train_data_X.shape[2]
        
    model = Sequential()
        # first lstm layer
    model.add(LSTM(units = space['units'], activation='tanh', input_shape=(train_X.shape[1],train_X.shape[2]), kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), return_sequences=True))

    if space['layers'] !=1:

        # building hidden layers
        for i in range(1, space['layers']):
            # for the last layer as the return sequence is False
            if i == space['layers'] -1:
                model.add(LSTM(int(space['units']/(2**i)),  activation='tanh', return_sequences=False))
            else:
                model.add(LSTM(int(space['units']/(2**i)),  activation='tanh', return_sequences=True))

    else:
        model.add(LSTM(int(space['units']/2),  activation='tanh', return_sequences=False))

    # adding dropout layer
    model.add(Dropout(space['dropout']))
    # final layer
    model.add(Dense(train_y.shape[1])) #train_data_y.shape[1]
    # metrics for evaluating the model
    metrics = [tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanAbsoluteError(), tf.keras.metrics.MeanAbsolutePercentageError()]

    # model compiler
    model.compile(optimizer=Adam(learning_rate = space['rate']), loss='mse', metrics = metrics)

    # setting the callback function
    # cb = [
    #     tf.keras.callbacks.ModelCheckpoint(path_checkpoint),
    #     tf.keras.callbacks.CSVLogger(path_metrics+'/'+'data.csv')]

    # model fitting protocol
    history = model.fit(train_X,train_y, 
                        epochs = 5, 
                        batch_size = 8,  
                        validation_split=0.1,
                        verbose = 1,
                        shuffle= False)

    # training dataset
    train_loss, RMSE, MAE, MAPE = model.evaluate(train_X,train_y)

    return {'loss': RMSE, 'status': STATUS_OK, 'model': model}


In [74]:
best_result = fmin(
    fn = build_model,
    space = space,
    algo = tpe.suggest,
    max_evals = 50,
    trials = Trials()
)

Epoch 1/5                                             

1/1 [==============================] - 6s 6s/step - loss: 5.0103 - root_mean_squared_error: 2.2382 - mean_absolute_error: 1.9942 - mean_absolute_percentage_error: 99.0646 - val_loss: 4.5948 - val_root_mean_squared_error: 2.1434 - val_mean_absolute_error: 1.8946 - val_mean_absolute_percentage_error: 92.9014

Epoch 2/5                                             

1/1 [==============================] - 0s 52ms/step - loss: 4.5948 - root_mean_squared_error: 2.1434 - mean_absolute_error: 1.8946 - mean_absolute_percentage_error: 92.9014 - val_loss: 4.1653 - val_root_mean_squared_error: 2.0408 - val_mean_absolute_error: 1.7878 - val_mean_absolute_percentage_error: 86.3886

Epoch 3/5                                             

1/1 [==============================] - 0s 48ms/step - loss: 4.1653 - root_mean_squared_error: 2.0408 - mean_absolute_error: 1.7878 - mean_absolute_percentage_error: 86.3886 - val_loss: 3.6749 - val_root_mean_squa

KeyboardInterrupt: 